<a href="https://colab.research.google.com/github/plundh/pl-dreambooth/blob/main/pl_ShivDreamBooth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Colab is based off of [Shivam Shriraro's repo](https://github.com/ShivamShrirao/diffusers/tree/main/examples/dreambooth), but with added convenience features.

*On your Google Drive:*
1. Put your training images in ***dreambooth/training_images/[TRAINING_FOLDER_NAME]***
2. Put your class images in ***dreambooth/class_images/[CLASS_FOLDER_NAME]***

# Notes
* This colab needs a GPU with 15 GB VRAM or more to run.
* Images: For objects and faces **10-30** training images are sufficient. For art styles, more are often better.
* Steps: **80-160** steps per training image
* That said, there is no reliable recipe at this point, so don't expect good results without plenty of experimentation.

In [ ]:
#@title Check Runtime GPU
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

In [ ]:
#@title #1. Settings

import os
from google.colab import drive

# Mount Google Drive if not already mounted
GDRIVE_PATH = "/content/google_drive"
if os.path.isdir(GDRIVE_PATH):
  print(f"Google Drive  already mounted at '{GDRIVE_PATH}'" )
else:
  drive.mount(GDRIVE_PATH)

#@markdown ###**Input Model**
#@markdown You have to: \\
#@markdown * Be a registered user in Hugging Face Hub and intput your [access token](https://huggingface.co/settings/tokens) here.
#@markdown * Accept the model license before downloading or using the Stable Diffusion weights. Visit the [model card](https://huggingface.co/runwayml/stable-diffusion-v1-5), read the license and tick the checkbox to agree.  
HUGGINGFACE_TOKEN = "" #@param {type:"string"}
INPUT_MODEL_NAME = "runwayml/stable-diffusion-v1-5" #@param ["runwayml/stable-diffusion-v1-5", "compvis/stable-diffusion-v1-4"]

#@markdown \
#@markdown ###**Training Images**
#@markdown _/content/google_drive/MyDrive/dreambooth/training_images/[TRAINING_FOLDER_NAME]_ \\
TRAINING_FOLDER_NAME = "azorn" #@param {type:"string"}
CONCEPT_NAME = "AndersZorn" #@param {type:"string"}
#@markdown If you wish to train multiple concepts, add additional dictionaries to **concepts_input** in the code. \\

#@markdown \
#@markdown ###**Class Images**
#@markdown _/content/google_drive/MyDrive/dreambooth/class_images/[CLASS_FOLDER_NAME]_ \\
#@markdown '*Prior Preservation*' uses generated reference images from the concept class to prevent your trained concept from distoring other instances of that class. \
#@markdown If no folder is provided, class images will be generated prior to training.
ENABLE_PRIOR_PRESERVATION = False #@param {type:"boolean"}
CLASS_FOLDER_NAME = "artstyle_ddim" #@param {type:"string"}

if ENABLE_PRIOR_PRESERVATION:
  CLASS_NAME = "artstyle" #@param {type:"string"}
else:
  CLASS_NAME = ""

TRAINING_IMAGES_ROOT = GDRIVE_PATH + "/MyDrive/dreambooth/training_images/"
CLASS_IMAGES_ROOT = GDRIVE_PATH + "/MyDrive/dreambooth/class_images/"

#@markdown \
#@markdown ###**Training Settings**
STEPS_PER_IMAGE = 120 #@param {type:"integer"}
STEPS_PER_IMAGE_SAVE_INTERVAL = 40 #@param {type:"integer"}

### CONCEPTS GO HERE #################################################################
# Add a new dictionary to 'concepts_input' for each extra concept you wish to train.

concepts_input = [
    {
        "CONCEPT_NAME":                   CONCEPT_NAME,
        "class_name":                     CLASS_NAME,
        "TRAINING_FOLDER_NAME":           TRAINING_FOLDER_NAME,
        "CLASS_FOLDER_NAME":              CLASS_FOLDER_NAME
#    },
#    {
#        "CONCEPT_NAME":                  "JohnSingerSargent",
#        "class_name":                    "artstyle",
#        "TRAINING_FOLDER_NAME":          "jsargent",
#        "CLASS_FOLDER_NAME":             "artstyle_ddim"
    }
]
######################################################################################

#@markdown \
#@markdown ###**Export**
#@markdown  Checkpoints will be saved as '**[date]** \_( **[token]** @ **[training_image_folder]** \_ **[image_count]** i)_ **[total images]** _ **{steps per total images}** .ckpt' \\
EXPORT_ONLY_LAST_CHECKPOINT = False #@param {type:"boolean"}

#@markdown \
#@markdown ###**Sample Generation**
#@markdown Will generate sample images to test how your model performs. A grid of varying 'steps' and 'CFG' settings will be generated for each checkpoint.
ENABLE_SAMPLE_GENERATION = False #@param {type:"boolean"}

DISCONNECT_ON_COMPLETION = False #@param {type:"boolean"}

In [ ]:
#@title #2. Verify Settings and Training Data

import math
import fnmatch
import json
from collections import Counter
from prettytable import PrettyTable
from prettytable import SINGLE_BORDER
from IPython.display import Markdown as md

concepts_list = []

for concept in concepts_input:
  instance_values = {
        "instance_prompt":      concept["CONCEPT_NAME"],
        "instance_data_dir":    f"{TRAINING_IMAGES_ROOT}{concept['TRAINING_FOLDER_NAME']}/",
        "inst_file_count":      len(os.listdir(f"{TRAINING_IMAGES_ROOT}{concept['TRAINING_FOLDER_NAME']}")),
     }
  
  if ENABLE_PRIOR_PRESERVATION:
    class_values ={
      "class_prompt":         concept["class_name"],
      "class_data_dir":       f"{CLASS_IMAGES_ROOT}{concept['CLASS_FOLDER_NAME']}/",
      "class_file_count":     len(os.listdir(f"{CLASS_IMAGES_ROOT}{concept['CLASS_FOLDER_NAME']}"))
    }
    combined_values = {**instance_values, **class_values}
    concepts_list.append(combined_values)
  else:
    concepts_list.append(instance_values)

if not ENABLE_PRIOR_PRESERVATION: print("☑️ Skipping Prior Preservation")

#print(json.dumps(concepts_list, sort_keys=True, indent=2)) # For debugging
#print("\n")

if len(concepts_list) == 1:
    print("Training 1 concept")
else:
    print(f"Training {len(concepts_list)} concepts")

for concept in concepts_list:
  print("\n")
  class_name = f"_{concept['class_prompt']}" if ENABLE_PRIOR_PRESERVATION else ''
  combined_token = ",".join([concept['instance_prompt']for concept in concepts_list])
  combined_token_class_folder = ",".join(["(" + str(concept['instance_prompt']) + class_name + "@" + str(os.path.basename(os.path.normpath(concept['instance_data_dir']))) + "_" + str(concept['inst_file_count']) + "i" + ")" for concept in concepts_list])
  if concept['inst_file_count'] == 0:
    print(f"❌ No training images found in '{concept['instance_data_dir']}'")
  else:
    print(f"✅ {concept['inst_file_count']} training images found in '{concept['instance_data_dir']}'")

  if ENABLE_PRIOR_PRESERVATION:
    if concept['class_file_count'] == 0:
          print(f"❌ No Class images found in '{concept['class_data_dir']}'")
    else:
      print(f"✅ {concept['class_file_count']} class images found in '{concept['class_data_dir']}'")

  concept_table = PrettyTable(header=False)
  concept_table.align = "l"
  concept_table.set_style(SINGLE_BORDER)

  for key, value in concept.items():
    concept_table.add_row([key, value])
  print(concept_table)

with open("concepts_list.json", "w") as f:
    json.dump(concepts_list, f, indent=4)

total_class_images = sum([concept['inst_file_count'] for concept in concepts_list])

## Output paths
total_training_images = sum([concept["inst_file_count"] for concept in concepts_list])
date_string = !date +"%Y-%m-%d_%H-%M"
training_steps = total_training_images * STEPS_PER_IMAGE
save_interval = total_training_images * STEPS_PER_IMAGE_SAVE_INTERVAL

def output_file_name(steps_per_image):
  return f"{date_string[-1]}_{combined_token_class_folder}_[{total_training_images}]_{{{steps_per_image}}}"

temp_folder = f"{date_string[-1]}_{combined_token_class_folder}_{training_steps}s"
temp_folder = temp_folder.replace(" ", "-")
temp_folder_root = f"stable_diffusion_weights/{temp_folder}"

!mkdir -p temp_folder
!mkdir -p "{GDRIVE_PATH}/MyDrive/dreambooth/models"

checkpoint_steps = training_steps
checkpoints_table = PrettyTable(['Step', 'Steps per image'])
checkpoints_table.set_style(SINGLE_BORDER)
checkpoints_table.sortby = "Step"

while checkpoint_steps > 0:
  steps_per_img = int(checkpoint_steps / total_training_images)
  checkpoints_table.add_row([checkpoint_steps, steps_per_img])
  checkpoint_steps -= STEPS_PER_IMAGE_SAVE_INTERVAL * total_training_images

print("\n")
print(f"Saving checkpoints every {STEPS_PER_IMAGE_SAVE_INTERVAL} steps per image:")
print(checkpoints_table)
print("\n")
print("Ready for training ...")

In [ ]:
#@title #3. Install Requirements

!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/examples/dreambooth/train_dreambooth.py
!wget -q https://github.com/ShivamShrirao/diffusers/raw/main/scripts/convert_diffusers_to_original_stable_diffusion.py
%pip install -qq git+https://github.com/ShivamShrirao/diffusers
%pip install -q -U --pre triton
%pip install -q accelerate==0.12.0 transformers ftfy bitsandbytes gradio natsort
%pip install -q https://github.com/metrolobo/xformers_wheels/releases/download/1d31a3ac_various_6/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl
# These were compiled on Tesla T4, should also work on P100, thanks to https://github.com/metrolobo

# If precompiled wheels don't work, install it with the following command. It will take around 40 minutes to compile.
# %pip install git+https://github.com/facebookresearch/xformers@1d31a3a#egg=xformers

!mkdir -p ~/.huggingface
!echo -n "{HUGGINGFACE_TOKEN}" > ~/.huggingface/token

print("Done!")

In [ ]:
#@title #4. Training

if ENABLE_PRIOR_PRESERVATION:
  !accelerate launch train_dreambooth.py \
    --pretrained_model_name_or_path="{INPUT_MODEL_NAME}" \
    --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
    --output_dir="{temp_folder_root}" \
    --with_prior_preservation \
    --prior_loss_weight=1.0 \
    --num_class_images=1000  \
    --seed=1337 \
    --resolution=512 \
    --train_batch_size=1 \
    --train_text_encoder \
    --mixed_precision="fp16" \
    --use_8bit_adam \
    --gradient_accumulation_steps=1 \
    --learning_rate=1e-6 \
    --lr_scheduler="constant" \
    --lr_warmup_steps=0 \
    --max_train_steps="{training_steps}" \
    --save_interval="{save_interval}" \
    --pad_tokens \
    --concepts_list="concepts_list.json"

else:
  !accelerate launch train_dreambooth.py \
    --pretrained_model_name_or_path="{INPUT_MODEL_NAME}" \
    --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
    --output_dir="{temp_folder_root}" \
    --seed=1337 \
    --resolution=512 \
    --train_batch_size=1 \
    --train_text_encoder \
    --mixed_precision="fp16" \
    --use_8bit_adam \
    --gradient_accumulation_steps=1 \
    --learning_rate=1e-6 \
    --lr_scheduler="constant" \
    --lr_warmup_steps=0 \
    --max_train_steps="{training_steps}" \
    --save_interval="{save_interval}" \
    --pad_tokens \
    --concepts_list="concepts_list.json"


print("Done!")

In [ ]:
#@title #5. Export Checkpoint files to Google Drive
from natsort import natsorted
from glob import glob
import os

half_arg = ""
fp16 = True
if fp16:
    half_arg = "--half"

def convert_diffusers(checkpoint_folder_name, steps_per_image):
  try:
    !python convert_diffusers_to_original_stable_diffusion.py --model_path "{checkpoint_folder_name}" --checkpoint_path "{output_file_name(steps_per_image)}" --half
    print(f"✅ Saved model '{output_file_name(steps_per_image)}'")
  except:
    print(f"❌ Could not save model '{output_file_name(training_steps)}'")

temp_folders_list = natsorted(glob(f"{temp_folder_root}/*/", recursive = True))

if not EXPORT_ONLY_LAST_CHECKPOINT:
  for folder in temp_folders_list:
    folder_checkpoint = int((os.path.basename(os.path.normpath(folder))))
    if folder_checkpoint not in [0, training_steps]:
      convert_diffusers(folder, int(folder_checkpoint/total_training_images))

convert_diffusers(temp_folders_list[-1], int(training_steps/total_training_images))

if ENABLE_PRIOR_PRESERVATION:
  print(f"To invoke your trained subject, use '{combined_token_class_folder}' in the prompt.")
else:
  print(f"To invoke your trained subject, use '{combined_token}' in the prompt.")

In [ ]:
#@title #5. Generate Sample Images
import torch
from torch import autocast
from PIL import Image
from diffusers import StableDiffusionPipeline, DDIMScheduler
from IPython.display import display
from google.colab import runtime

g_cuda = torch.Generator(device='cuda')
scheduler = DDIMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear", clip_sample=False, set_alpha_to_one=False)

def image_grid(imgs, rows, cols):
    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size\
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

if ENABLE_SAMPLE_GENERATION:
  SAMPLES_ROOT = f"{GDRIVE_PATH}/MyDrive/dreambooth/models/samples"
  !mkdir -p "{samples_root}"
  checkpoints_list = [folder for folder in natsorted(glob(f"{temp_folder_root}/*/", recursive = True)) if int((os.path.basename(os.path.normpath(folder)))) != 0]
  prompt = f"{CONCEPT_NAME} painting"

  sample_steps = [5,10,20,40,60,80,100,120,150]
  sample_CFG = [3, 6, 9, 12, 15, 18, 21]

  images = []

  for folder in checkpoints_list:
    pipe = StableDiffusionPipeline.from_pretrained(folder, scheduler=scheduler, safety_checker=None, torch_dtype=torch.float16).to("cuda")
    checkpoint_name = int((os.path.basename(os.path.normpath(folder))))
    for steps in sample_steps:
      for CFG in sample_CFG:
        with autocast("cuda"), torch.inference_mode():
          image = pipe(
              prompt,
              height=640,
              width=512,
              num_images_per_prompt=1,
              num_inference_steps=steps,
              guidance_scale=CFG,
              generator=g_cuda
          ).images
          images = images + image

    grid = image_grid(images, rows=len(sample_CFG), cols=len(sample_steps))
    file_path = f'{SAMPLES_ROOT}/{output_file_name(int(checkpoint_name/total_training_images))}.jpg'
    grid.save(file_path)
    print(f'Saved {file_path}')

if DISCONNECT_ON_COMPLETION:
  runtime.unassign()